# Links úteis para se usar datas para treinar modelo 
https://towardsdatascience.com/machine-learning-with-datetime-feature-engineering-predicting-healthcare-appointment-no-shows-5e4ca3a85f96

https://stackoverflow.com/questions/16453644/regression-with-date-variable-using-scikit-learn


### Para treinar o modelo fazer onehot encoding para distritos e concelhos e ignorar as localidas (para não ficarmos com um nº ridiculo de colunas)

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
import sklearn.metrics as metrics

In [2]:
data = pd.read_csv('data/anpc_final.csv')

C:\Users\tiago\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 573971 entries, 0 to 573970
Data columns (total 15 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   Numero                                  573971 non-null  object 
 1   DataOcorrencia                          573966 non-null  object 
 2   DataFechoOperacional                    418785 non-null  object 
 3   Natureza                                573966 non-null  object 
 4   EstadoOcorrencia                        573966 non-null  object 
 5   Distrito                                573966 non-null  object 
 6   Concelho                                573966 non-null  object 
 7   Freguesia                               573964 non-null  object 
 8   Localidade                              573964 non-null  object 
 9   Latitude                                573962 non-null  object 
 10  Longitude                               5739

In [4]:
data.describe()

,NumeroMeiosTerrestresEnvolvidos,NumeroOperacionaisTerrestresEnvolvidos,NumeroMeiosAereosEnvolvidos,NumeroOperacionaisAereosEnvolvidos
count,573962.000000,573962.000000,573962.000000,573962.000000
mean,1.522423,4.114807,0.018506,0.025385
std,2.303219,8.293575,0.220632,0.351961
min,0.000000,0.000000,0.000000,0.000000
25%,1.000000,2.000000,0.000000,0.000000
50%,1.000000,2.000000,0.000000,0.000000
75%,1.000000,4.000000,0.000000,0.000000
max,354.000000,2170.000000,41.000000,61.000000


In [5]:
data.head(3)

,Numero,DataOcorrencia,DataFechoOperacional,Natureza,EstadoOcorrencia,Distrito,Concelho,Freguesia,Localidade,Latitude,Longitude,NumeroMeiosTerrestresEnvolvidos,NumeroOperacionaisTerrestresEnvolvidos,NumeroMeiosAereosEnvolvidos,NumeroOperacionaisAereosEnvolvidos
0,2016150002576,09/01/2016 14:02:00,09/01/2016 17:30:00,Protecção e Assistência a Pessoas e Bens / Ass...,Encerrada,SETÚBAL,MOITA,Gaio-Rosário e Sarilhos Pequenos,Gaio- Rosario,"38,68091202","-9,002235449",1.0,3.0,0.0,0.0
1,2016010098470,10/01/2016 00:30:00,NaN,Protecção e Assistência a Pessoas e Bens / Ass...,Em Curso,AVEIRO,ALBERGARIA-A-VELHA,Albergaria-a-Velha e Valmaior,Rua 25 de Abril - Assilhó - Albergaria-a-Velha,"40,6927855","-8,479686736",0.0,0.0,0.0,0.0
2,2016020002249,24/01/2016 18:57:00,24/01/2016 19:53:00,Protecção e Assistência a Pessoas e Bens / Ass...,Encerrada,BEJA,CASTRO VERDE,Castro Verde e Casével,"CASTRO VERDE-Rua Bento J. Carraça,6","37,71278977","-8,091107843",2.0,4.0,0.0,0.0


In [6]:
# Temos que ver se conmpensa mais usar as 'Naturezas' juntas ou separar 
data.Natureza.unique().shape

(105,)

In [7]:
#Número de ocorencias por distrito

data.groupby(['Distrito']).size().sort_values(ascending=False)

Distrito
LISBOA              97695
PORTO               93203
BRAGA               42017
SETÚBAL             41425
AVEIRO              41361
SANTARÉM            33464
VISEU               30291
FARO                29947
LEIRIA              29685
COIMBRA             25495
VILA REAL           17934
BRAGANÇA            15863
VIANA DO CASTELO    15282
GUARDA              13423
CASTELO BRANCO      12912
BEJA                12011
PORTALEGRE          11550
ÉVORA               10406
0                       2
dtype: int64

In [8]:
data.Distrito.isna().sum()

5

In [9]:
data[data['Distrito'] == '0']

,Numero,DataOcorrencia,DataFechoOperacional,Natureza,EstadoOcorrencia,Distrito,Concelho,Freguesia,Localidade,Latitude,Longitude,NumeroMeiosTerrestresEnvolvidos,NumeroOperacionaisTerrestresEnvolvidos,NumeroMeiosAereosEnvolvidos,NumeroOperacionaisAereosEnvolvidos
83938,LAR SAO JOSE,"39,5983240486353","-8,39819137470253",1,2,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
262081,Travessa da Água da Flor - Frent PSP,"38,71853771","-9,165754765",1,2,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
#Retirar ocorrências sem distrito definido
data.drop(data[data['Distrito'] == '0'].index, inplace=True)

In [11]:
data.groupby(['Distrito']).size().sort_values(ascending=False)

Distrito
LISBOA              97695
PORTO               93203
BRAGA               42017
SETÚBAL             41425
AVEIRO              41361
SANTARÉM            33464
VISEU               30291
FARO                29947
LEIRIA              29685
COIMBRA             25495
VILA REAL           17934
BRAGANÇA            15863
VIANA DO CASTELO    15282
GUARDA              13423
CASTELO BRANCO      12912
BEJA                12011
PORTALEGRE          11550
ÉVORA               10406
dtype: int64

In [12]:
##HISTOGRAMA NÚMERO DE OCORRêNCIAS POR DISTRITO## 

In [13]:
# Mostrar ocorrências por distrito num mapa (ideia possível e que o sérgio vai gostar, mas temos que discutir isto melhor)

In [14]:
data.DataOcorrencia = pd.to_datetime(data.DataOcorrencia)
data.DataOcorrencia.dt.year.unique()

array([2016.,   nan, 2017., 2018.])

In [15]:
data.DataFechoOperacional = pd.to_datetime(data.DataFechoOperacional)

In [16]:
data.head()

,Numero,DataOcorrencia,DataFechoOperacional,Natureza,EstadoOcorrencia,Distrito,Concelho,Freguesia,Localidade,Latitude,Longitude,NumeroMeiosTerrestresEnvolvidos,NumeroOperacionaisTerrestresEnvolvidos,NumeroMeiosAereosEnvolvidos,NumeroOperacionaisAereosEnvolvidos
0,2016150002576,2016-09-01 14:02:00,2016-09-01 17:30:00,Protecção e Assistência a Pessoas e Bens / Ass...,Encerrada,SETÚBAL,MOITA,Gaio-Rosário e Sarilhos Pequenos,Gaio- Rosario,"38,68091202","-9,002235449",1.0,3.0,0.0,0.0
1,2016010098470,2016-10-01 00:30:00,NaT,Protecção e Assistência a Pessoas e Bens / Ass...,Em Curso,AVEIRO,ALBERGARIA-A-VELHA,Albergaria-a-Velha e Valmaior,Rua 25 de Abril - Assilhó - Albergaria-a-Velha,"40,6927855","-8,479686736",0.0,0.0,0.0,0.0
2,2016020002249,2016-01-24 18:57:00,2016-01-24 19:53:00,Protecção e Assistência a Pessoas e Bens / Ass...,Encerrada,BEJA,CASTRO VERDE,Castro Verde e Casével,"CASTRO VERDE-Rua Bento J. Carraça,6","37,71278977","-8,091107843",2.0,4.0,0.0,0.0
3,2016050041560,2016-01-30 23:38:00,NaT,Protecção e Assistência a Pessoas e Bens / Ass...,Em Curso,CASTELO BRANCO,VILA DE REI,Vila de Rei,Vila de Rei,"39,6684201","-8,146938534",1.0,2.0,0.0,0.0
4,2016020003825,2016-02-13 00:10:00,2016-02-13 01:35:00,Protecção e Assistência a Pessoas e Bens / Ass...,Encerrada,BEJA,CASTRO VERDE,Santa Bárbara de Padrões,"SETE-Rua da Graça, 4","37,6149714","-7,962115173",1.0,2.0,0.0,0.0


In [47]:
data.Natureza.unique()

array(['Protecção e Assistência a Pessoas e Bens / Assistência e Prevenção a actividades humanas / Prevenção a actividades de lazer',
       'Protecção e Assistência a Pessoas e Bens / Assistência e Prevenção a actividades humanas / Limpeza de Via e Sinalização de Perigo',
       'Protecção e Assistência a Pessoas e Bens / Assistência em Saúde / Intoxicação',
       'Protecção e Assistência a Pessoas e Bens / Assistência em Saúde / Trauma',
       'Riscos Mistos / Incêndios Rurais / Povoamento Florestal',
       'Riscos Tecnológicos / Incêndios Urbanos ou em Área Urbanizável / Habitacional',
       'Riscos Mistos / Incêndios Rurais / Mato',
       'Riscos Mistos / Incêndios Rurais / Gestão de Combustível',
       'Riscos Tecnológicos / Acidentes / Colisão rodoviária',
       'Riscos Mistos / Comprometimento total ou parcial de segurança, serviços ou estruturas / Queda de Árvore',
       'Riscos Tecnológicos / Acidentes / Atropelamento rodoviário',
       'Riscos Tecnológicos / Acidente

In [17]:
# Creating the column representing the ammout of time necessary to solve each occurrence
data['TempoResolucao'] = (data.DataFechoOperacional - data.DataOcorrencia)
data.head()

,Numero,DataOcorrencia,DataFechoOperacional,Natureza,EstadoOcorrencia,Distrito,Concelho,Freguesia,Localidade,Latitude,Longitude,NumeroMeiosTerrestresEnvolvidos,NumeroOperacionaisTerrestresEnvolvidos,NumeroMeiosAereosEnvolvidos,NumeroOperacionaisAereosEnvolvidos,TempoResolucao
0,2016150002576,2016-09-01 14:02:00,2016-09-01 17:30:00,Protecção e Assistência a Pessoas e Bens / Ass...,Encerrada,SETÚBAL,MOITA,Gaio-Rosário e Sarilhos Pequenos,Gaio- Rosario,"38,68091202","-9,002235449",1.0,3.0,0.0,0.0,0 days 03:28:00
1,2016010098470,2016-10-01 00:30:00,NaT,Protecção e Assistência a Pessoas e Bens / Ass...,Em Curso,AVEIRO,ALBERGARIA-A-VELHA,Albergaria-a-Velha e Valmaior,Rua 25 de Abril - Assilhó - Albergaria-a-Velha,"40,6927855","-8,479686736",0.0,0.0,0.0,0.0,NaT
2,2016020002249,2016-01-24 18:57:00,2016-01-24 19:53:00,Protecção e Assistência a Pessoas e Bens / Ass...,Encerrada,BEJA,CASTRO VERDE,Castro Verde e Casével,"CASTRO VERDE-Rua Bento J. Carraça,6","37,71278977","-8,091107843",2.0,4.0,0.0,0.0,0 days 00:56:00
3,2016050041560,2016-01-30 23:38:00,NaT,Protecção e Assistência a Pessoas e Bens / Ass...,Em Curso,CASTELO BRANCO,VILA DE REI,Vila de Rei,Vila de Rei,"39,6684201","-8,146938534",1.0,2.0,0.0,0.0,NaT
4,2016020003825,2016-02-13 00:10:00,2016-02-13 01:35:00,Protecção e Assistência a Pessoas e Bens / Ass...,Encerrada,BEJA,CASTRO VERDE,Santa Bárbara de Padrões,"SETE-Rua da Graça, 4","37,6149714","-7,962115173",1.0,2.0,0.0,0.0,0 days 01:25:00


In [45]:
# Checking for negative resolution times
data[data.TempoResolucao.dt.days < 0]

,Numero,DataOcorrencia,DataFechoOperacional,Natureza,EstadoOcorrencia,Distrito,Concelho,Freguesia,Localidade,Latitude,Longitude,NumeroMeiosTerrestresEnvolvidos,NumeroOperacionaisTerrestresEnvolvidos,NumeroMeiosAereosEnvolvidos,NumeroOperacionaisAereosEnvolvidos,TempoResolucao


In [19]:
# Removing negative times of resolution
data.drop(data[data.TempoResolucao.dt.days < 0].index, inplace=True)

In [20]:
data.Natureza.isna().sum()
data[data.Natureza.isna()]

,Numero,DataOcorrencia,DataFechoOperacional,Natureza,EstadoOcorrencia,Distrito,Concelho,Freguesia,Localidade,Latitude,Longitude,NumeroMeiosTerrestresEnvolvidos,NumeroOperacionaisTerrestresEnvolvidos,NumeroMeiosAereosEnvolvidos,NumeroOperacionaisAereosEnvolvidos,TempoResolucao
83933,\t\t\t\t\t\t\t\t,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
83934,10.15\tTomar\tIPO\t\t\t\tS\t\tS,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
83935,11.30\tTomar\tIPO\t\t\t\tS\t\tN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
83936,\t\t\t\t\t\t\t\t,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
83937,13.15\tTomar\tIPO\t\t\t\tS\t\tS,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [21]:
# Droping useless rows
data.dropna(subset=['Natureza'], inplace=True)
data[data.Natureza.isna()]


,Numero,DataOcorrencia,DataFechoOperacional,Natureza,EstadoOcorrencia,Distrito,Concelho,Freguesia,Localidade,Latitude,Longitude,NumeroMeiosTerrestresEnvolvidos,NumeroOperacionaisTerrestresEnvolvidos,NumeroMeiosAereosEnvolvidos,NumeroOperacionaisAereosEnvolvidos,TempoResolucao


In [22]:
data.EstadoOcorrencia.unique()

array(['Encerrada', 'Em Curso', 'Falso Alarme', 'Falso Alerta',
       'Conclusão', 'Despacho', 'Despacho de 1º Alerta', 'Em Resolução',
       'Chegada ao TO', 'Vigilância'], dtype=object)

In [42]:
data[data.EstadoOcorrencia == 'Em Curso']
# Podemos incluir isto para a análise a nível de ocorrências por distrito e assim, mas depois temos que dar drop disto
# temos que discutir isto melhor

,Numero,DataOcorrencia,DataFechoOperacional,Natureza,EstadoOcorrencia,Distrito,Concelho,Freguesia,Localidade,Latitude,Longitude,NumeroMeiosTerrestresEnvolvidos,NumeroOperacionaisTerrestresEnvolvidos,NumeroMeiosAereosEnvolvidos,NumeroOperacionaisAereosEnvolvidos,TempoResolucao
1,2016010098470,2016-10-01 00:30:00,NaT,Protecção e Assistência a Pessoas e Bens / Ass...,Em Curso,AVEIRO,ALBERGARIA-A-VELHA,Albergaria-a-Velha e Valmaior,Rua 25 de Abril - Assilhó - Albergaria-a-Velha,"40,6927855","-8,479686736",0.0,0.0,0.0,0.0,NaT
3,2016050041560,2016-01-30 23:38:00,NaT,Protecção e Assistência a Pessoas e Bens / Ass...,Em Curso,CASTELO BRANCO,VILA DE REI,Vila de Rei,Vila de Rei,"39,6684201","-8,146938534",1.0,2.0,0.0,0.0,NaT
119819,2016080056628,2016-12-29 18:56:00,NaT,Protecção e Assistência a Pessoas e Bens / Ass...,Em Curso,FARO,ALBUFEIRA,Albufeira e Olhos de Água,ALBUFEIRA,"37,09172682","-8,252292264",1.0,2.0,0.0,0.0,NaT
119823,2016090039202,2016-12-29 18:53:00,NaT,Protecção e Assistência a Pessoas e Bens / Ass...,Em Curso,GUARDA,SEIA,Pinhanços,Lar Pinhanços,"40,46942478","-7,688956785",1.0,2.0,0.0,0.0,NaT
119842,2016080056622,2016-12-29 17:52:00,NaT,Protecção e Assistência a Pessoas e Bens / Ass...,Em Curso,FARO,OLHÃO,Olhão,OLHÃO,"37,02205442","-7,843132514",1.0,2.0,0.0,0.0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573939,2018150114444,2018-12-31 03:57:00,NaT,Protecção e Assistência a Pessoas e Bens / Ass...,Em Curso,SETÚBAL,SEIXAL,Amora,Amora,"38,6138797308902","-9,12886309841698",1.0,2.0,0.0,0.0,NaT
573948,2018150114435,2018-12-31 03:18:00,NaT,Protecção e Assistência a Pessoas e Bens / Ass...,Em Curso,SETÚBAL,GRÂNDOLA,Grândola e Santa Margarida da Serra,GRANDOLA,"38,1825915644394","-8,56071313896018",1.0,2.0,0.0,0.0,NaT
573949,2018100076402,2018-12-31 03:08:00,NaT,Protecção e Assistência a Pessoas e Bens / Ass...,Em Curso,LEIRIA,LEIRIA,Monte Redondo e Carreira,União das Freguesias de Monte Redondo e Carreira,"39,8947498942488","-8,83974172991636",1.0,2.0,0.0,0.0,NaT
573953,2018150114431,2018-12-31 02:41:00,NaT,Protecção e Assistência a Pessoas e Bens / Ass...,Em Curso,SETÚBAL,ALMADA,"Almada, Cova da Piedade, Pragal e Cacilhas",Cacilhas,"38,6752676774718","-9,1644981889879",0.0,0.0,0.0,0.0,NaT


# Começar a experimentar com o modelo

In [24]:
enc = OneHotEncoder()
scaler = MaxAbsScaler()

In [25]:
data_modelo = data.copy()
data_modelo.TempoResolucao = data_modelo.TempoResolucao.dt.total_seconds()
# converter novamente para dias
#data.TempoResolucao = pd.to_timedelta(data.TempoResolucao, unit='s')
data_modelo.drop(columns=['Numero','DataOcorrencia', 'DataFechoOperacional', 'EstadoOcorrencia', 'Freguesia', 'Localidade'], inplace=True)
data_modelo.head()

,Natureza,Distrito,Concelho,Latitude,Longitude,NumeroMeiosTerrestresEnvolvidos,NumeroOperacionaisTerrestresEnvolvidos,NumeroMeiosAereosEnvolvidos,NumeroOperacionaisAereosEnvolvidos,TempoResolucao
0,Protecção e Assistência a Pessoas e Bens / Ass...,SETÚBAL,MOITA,"38,68091202","-9,002235449",1.0,3.0,0.0,0.0,12480.0
1,Protecção e Assistência a Pessoas e Bens / Ass...,AVEIRO,ALBERGARIA-A-VELHA,"40,6927855","-8,479686736",0.0,0.0,0.0,0.0,NaN
2,Protecção e Assistência a Pessoas e Bens / Ass...,BEJA,CASTRO VERDE,"37,71278977","-8,091107843",2.0,4.0,0.0,0.0,3360.0
3,Protecção e Assistência a Pessoas e Bens / Ass...,CASTELO BRANCO,VILA DE REI,"39,6684201","-8,146938534",1.0,2.0,0.0,0.0,NaN
4,Protecção e Assistência a Pessoas e Bens / Ass...,BEJA,CASTRO VERDE,"37,6149714","-7,962115173",1.0,2.0,0.0,0.0,5100.0


In [26]:
data_modelo.isna().sum()

Natureza                                       0
Distrito                                       0
Concelho                                       0
Latitude                                       2
Longitude                                      2
NumeroMeiosTerrestresEnvolvidos                2
NumeroOperacionaisTerrestresEnvolvidos         2
NumeroMeiosAereosEnvolvidos                    2
NumeroOperacionaisAereosEnvolvidos             2
TempoResolucao                            155181
dtype: int64

In [27]:
data_modelo.dropna(inplace=True)
data_modelo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 417694 entries, 0 to 573970
Data columns (total 10 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   Natureza                                417694 non-null  object 
 1   Distrito                                417694 non-null  object 
 2   Concelho                                417694 non-null  object 
 3   Latitude                                417694 non-null  object 
 4   Longitude                               417694 non-null  object 
 5   NumeroMeiosTerrestresEnvolvidos         417694 non-null  float64
 6   NumeroOperacionaisTerrestresEnvolvidos  417694 non-null  float64
 7   NumeroMeiosAereosEnvolvidos             417694 non-null  float64
 8   NumeroOperacionaisAereosEnvolvidos      417694 non-null  float64
 9   TempoResolucao                          417694 non-null  float64
dtypes: float64(5), object(5)
memory usage: 35.1+

In [28]:
X = data_modelo.drop(columns=['TempoResolucao']).copy()
y = data_modelo.TempoResolucao.copy()

In [29]:
X = enc.fit_transform(X)
X = scaler.fit_transform(X)
X

<417694x157410 sparse matrix of type '<class 'numpy.float64'>'
	with 3759246 stored elements in Compressed Sparse Row format>

In [30]:
y = y.ravel()

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

reg = LinearRegression().fit(X_train, y_train)

pred_y = reg.predict(X_test)

In [32]:

print("The first five prediction {}".format(pred_y[:5]))
print("The real first five labels {}".format(y_test[:5]))

The first five prediction [116569.20766943   4937.63222523   3030.63159051  58788.08876374
  36404.07664936]
The real first five labels [8880. 2760. 5400. 3480. 5760.]


In [33]:
mse = metrics.mean_squared_error(y_test, pred_y)
print(f"Mean Squared Error {mse}")

Mean Squared Error 405315225644.8157


## Outra versao para depois comparar, não é relevante por agora

In [34]:
data_v2 = data.copy()
data_v2['Natureza'] = data_v2['Natureza'].apply(lambda x: x.split(' / '))

In [35]:
columns = pd.DataFrame(data_v2['Natureza'].tolist())
columns


,0,1,2,3
0,Protecção e Assistência a Pessoas e Bens,Assistência e Prevenção a actividades humanas,Prevenção a actividades de lazer,None
1,Protecção e Assistência a Pessoas e Bens,Assistência e Prevenção a actividades humanas,Limpeza de Via e Sinalização de Perigo,None
2,Protecção e Assistência a Pessoas e Bens,Assistência em Saúde,Intoxicação,None
3,Protecção e Assistência a Pessoas e Bens,Assistência em Saúde,Trauma,None
4,Protecção e Assistência a Pessoas e Bens,Assistência em Saúde,Intoxicação,None
...,...,...,...,...
572872,Riscos Mistos,Incêndios Rurais,Povoamento Florestal,None
572873,Protecção e Assistência a Pessoas e Bens,Assistência em Saúde,Intoxicação,None
572874,Protecção e Assistência a Pessoas e Bens,Assistência em Saúde,Trauma,None
572875,Protecção e Assistência a Pessoas e Bens,Assistência em Saúde,Trauma,None


In [65]:
columns[0].unique()

array(['Protecção e Assistência a Pessoas e Bens', 'Riscos Mistos',
       'Riscos Tecnológicos', 'Riscos Naturais',
       'Operações  e Estados de Alerta'], dtype=object)

In [64]:
columns[1].unique()

array(['Assistência e Prevenção a actividades humanas',
       'Assistência em Saúde', 'Incêndios Rurais',
       'Incêndios Urbanos ou em Área Urbanizável', 'Acidentes',
       'Comprometimento total ou parcial de segurança, serviços ou estruturas',
       'Incêndios em Transportes', 'Intervenção em conflitos legais',
       'Acidentes industriais e tecnológicos', 'Incêndios em Detritos',
       'Incêndios em Equipamento e Produtos', 'Fenómenos Naturais',
       'Operações'], dtype=object)

In [63]:
columns[2].unique()

array(['Prevenção a actividades de lazer',
       'Limpeza de Via e Sinalização de Perigo', 'Intoxicação', 'Trauma',
       'Povoamento Florestal', 'Habitacional', 'Mato',
       'Gestão de Combustível', 'Colisão rodoviária', 'Queda de Árvore',
       'Atropelamento rodoviário', 'Despiste',
       'Busca e Resgate Terrestre, de Animais',
       'Abastecimento de Água à População',
       'Dano em redes de abastecimento de água',
       'Dano ou Queda de redes de fornecimento eléctrico', 'Rodoviário',
       'Queda de elementos de construção em estruturas edificadas',
       'Abastecimento de Água a entidades privadas', 'Agrícola',
       'Corte ou Remoção de elementos em perigo de queda',
       'Agressão/Violação', 'Dano em redes de abastecimento de gás',
       'Acidentes com veículos fora de estrada',
       'Inundação de estruturas por água canalizada',
       'Patrulhamento, Reconhecimento e Vigilância',
       'Fuga de Gás em garrafa',
       'Inundação de estruturas ou superfíci

In [62]:
columns[3].unique()

array([None, 'Tamponamento'], dtype=object)

## Acho que deveriamos retirar coluna 3 e mantinha separadas. Podemos prever separadamente, vamos conseguir mostrar bons resultados pelo menos na coluna 0 e 1 
Coluna 0 - 5 resultados únicos

Coluna 1 - 13 resultados únicos

Coluna 2 - 103 resultados únicos

In [36]:
new_columns = np.concatenate((columns[0].unique(),columns[1].unique(),columns[2].unique(),columns[3].unique()))
new_columns = new_columns[new_columns != np.array(None)]
new_columns = new_columns.tolist()
new_columns

['Protecção e Assistência a Pessoas e Bens',
 'Riscos Mistos',
 'Riscos Tecnológicos',
 'Riscos Naturais',
 'Operações  e Estados de Alerta',
 'Assistência e Prevenção a actividades humanas',
 'Assistência em Saúde',
 'Incêndios Rurais',
 'Incêndios Urbanos ou em Área Urbanizável',
 'Acidentes',
 'Comprometimento total ou parcial de segurança, serviços ou estruturas',
 'Incêndios em Transportes',
 'Intervenção em conflitos legais',
 'Acidentes industriais e tecnológicos',
 'Incêndios em Detritos',
 'Incêndios em Equipamento e Produtos',
 'Fenómenos Naturais',
 'Operações',
 'Prevenção a actividades de lazer',
 'Limpeza de Via e Sinalização de Perigo',
 'Intoxicação',
 'Trauma',
 'Povoamento Florestal',
 'Habitacional',
 'Mato',
 'Gestão de Combustível',
 'Colisão rodoviária',
 'Queda de Árvore',
 'Atropelamento rodoviário',
 'Despiste',
 'Busca e Resgate Terrestre, de Animais',
 'Abastecimento de Água à População',
 'Dano em redes de abastecimento de água',
 'Dano ou Queda de redes de 

In [37]:
data_v2 = data_v2.assign(**dict.fromkeys(new_columns, 0))

Dados entre os anos 2016 e 2018 - comparar com populção por distrito 

In [38]:
pd.read_excel('data/pop.xlsx')

,Nome,Divisão Administrativa,População Residente,População/km2,Ano*
0,Lisboa,Distrito Administrativo,2275591,824.2,2021
1,Porto,Distrito Administrativo,1786656,746.0,2021
2,Setúbal,Distrito Administrativo,875656,172.9,2021
3,Braga,Distrito Administrativo,846515,312.8,2021
4,Aveiro,Distrito Administrativo,700964,250.5,2021
5,Faro,Distrito Administrativo,467495,94.6,2021
6,Leiria,Distrito Administrativo,458679,130.9,2021
7,Santarém,Distrito Administrativo,425431,67.9,2021
8,Coimbra,Distrito Administrativo,408631,103.5,2021
9,Viseu,Distrito Administrativo,351592,70.2,2021


## Useful snippet to use later when dealing with training the model

In [39]:
# Convert to time in seconds to use as regression label
x = data.TempoResolucao
y = x.dt.total_seconds()
# Convert back to datetime object
z = pd.to_timedelta(y, unit='s')
z


0        0 days 03:28:00
1                    NaT
2        0 days 00:56:00
3                    NaT
4        0 days 01:25:00
               ...      
573966   0 days 00:53:00
573967   0 days 01:41:00
573968   0 days 02:48:00
573969   0 days 01:46:00
573970   0 days 23:59:00
Name: TempoResolucao, Length: 572877, dtype: timedelta64[ns]